In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'app')))

print(sys.path)

['/usr/lib64/python39.zip', '/usr/lib64/python3.9', '/usr/lib64/python3.9/lib-dynload', '', '/home/scur0279/MMA/.venv/lib64/python3.9/site-packages', '/home/scur0279/MMA/.venv/lib/python3.9/site-packages', '/gpfs/home5/scur0279/MMA/app']


In [2]:
from db.database import Database
import uuid

/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/scur0279/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/scur0279/MMA/.venv/lib64/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/scur0279/MMA/.venv/lib64/python3.9/site-packages/lpips/weights/v0.1/alex.pth


In [3]:
ls

create.py    example.ipynb  __init__.py  __pycache__/
database.py  images/        mma.db


#### stuff for testing the saving

In [4]:
from modules.prompt import Prompt
from modules.prompt_image import PromptImage
from PIL import Image

# Step 1: Reset DB and insert users/chats (keep as is)
datab = Database()
datab.connect()
datab.reset_tables()

# --- Step 1: Create Users and Create Chats ---
datab.insert_user("first_test_user", "password123")
datab.insert_user("second_test_user", "password456")

datab.insert_chat("First Test Chat", 1)
datab.insert_chat("Second Test Chat", 2)

chat_id_1=1
chat_id_2=2

# Step 2: Generate Prompt instances
# Prompy inputs are: prompt: str, depth: int, input_image: PromptImage = None, suggestion_used: str = None, modified_suggestion: bool = False):
prompt_1 = Prompt(prompt="Generate a bottle of perfume", depth=1)
prompt_2 = Prompt(prompt="Generate a bottle of perfume in a desert", depth=2)
prompt_3 = Prompt(prompt="Add sunset lighting to the perfume bottle scene", depth=3, suggestion_used="Try sunset colors", modified_suggestion = False)

# Manually attach the extra attributes
prompt_3.is_enhanced = True
prompt_3.enhanced_prompt = "A perfume bottle in a desert with glowing sunset hues"

# Save prompts through save_prompt() instead of insert_prompt()
datab.save_prompt(prompt_1, chat_id_1, user_id=1)
datab.save_prompt(prompt_2, chat_id_1, user_id=1)
datab.save_prompt(prompt_3, chat_id_1, user_id=1)

# Step 3: Generate and save images
# image_path_1 = "app/db/images/sauvage.png"
# image_path_2 = "app/db/images/sauvage_desert.png"
image_path_1 = "images/sauvage.png"
image_path_2 = "images/sauvage_desert.png"

image_1 = Image.open(image_path_1)
image_2 = Image.open(image_path_2)

prompt_image_1 = PromptImage(image=image_1, prompt_guidance=6.0, image_guidance=0.4,
                             input_prompt=prompt_1.id, output_prompt=prompt_1.id, save=False)
prompt_image_2 = PromptImage(image=image_2, prompt_guidance=7.0, image_guidance=0.5,
                             input_prompt=prompt_2.id, output_prompt=prompt_2.id, save=False)

datab.save_image(prompt_image_1, session_id=chat_id_1, user_id=1)
datab.save_image(prompt_image_2, session_id=chat_id_1, user_id=1)


# Adding data
# User 1: New Chat
datab.connect()
chat_id_3 = datab.insert_chat("User 1 - Extra Chat", 1)

extra_prompt_1 = Prompt(prompt="Design a futuristic drone", depth=1)
extra_prompt_2 = Prompt(prompt="Add neon lighting to the drone", depth=2)
extra_prompt_3 = Prompt(prompt="Place the drone in a cyberpunk city", depth=3)
extra_prompt_3.used_suggestion = True
extra_prompt_3.suggestion_used = "Try placing it in a city"
extra_prompt_3.modified_suggestion = True

datab.save_prompt(extra_prompt_1, chat_id_3, user_id=1)
datab.save_prompt(extra_prompt_2, chat_id_3, user_id=1)
datab.save_prompt(extra_prompt_3, chat_id_3, user_id=1)

image_3 = Image.open("images/test_image.jpg")
image_4 = Image.open("images/test_image2.jpg")
image_5 = Image.open("images/test_image3.jpg")


prompt_image_3 = PromptImage(image=image_3, prompt_guidance=5.5, image_guidance=0.3,
                             input_prompt=extra_prompt_1.id, output_prompt=extra_prompt_1.id, save=False)
prompt_image_4 = PromptImage(image=image_4, prompt_guidance=6.5, image_guidance=0.45,
                             input_prompt=extra_prompt_3.id, output_prompt=extra_prompt_3.id, save=False)
prompt_image_5 = PromptImage(image=image_5, prompt_guidance=6.5, image_guidance=0.45,
                             input_prompt=extra_prompt_2.id, output_prompt=extra_prompt_2.id, save=False)

datab.save_image(prompt_image_3, session_id=chat_id_3, user_id=1)
datab.save_image(prompt_image_4, session_id=chat_id_3, user_id=1)
datab.save_image(prompt_image_5, session_id=chat_id_3, user_id=1)

# User 2: Chat with Depth 5
datab.connect()
chat_id_4 = datab.insert_chat("User 2 - Long Chain", 2)

p1 = Prompt("A spaceship flying through clouds", depth=1)
p2 = Prompt("Add sunlight breaking through clouds", depth=2)
p3 = Prompt("Include a fleet of ships in the distance", depth=3)
p4 = Prompt("Make the ships look metallic", depth=4, suggestion_used="Make them metallic", modified_suggestion=False)
p5 = Prompt("Add reflections from the sun", depth=5)
p5.is_enhanced = True
p5.enhanced_prompt = "Fleet of metallic ships reflecting golden sunlight"

for p in [p1, p2, p3, p4, p5]:
    datab.save_prompt(p, chat_id_4, user_id=2)

img_paths = [
    "images/cat.png",
    "images/cat1.png",
    "images/cat2.png",
    "images/cat3.png",
    "images/cat4.png"
]

guidances = [(6.0, 0.4), (6.5, 0.45), (7.0, 0.5), (7.5, 0.55)]
prompts = [p1, p2, p3, p4, p5]

for path, (pg, ig), prompt in zip(img_paths, guidances, prompts):
    img = Image.open(path)
    pimg = PromptImage(image=img, prompt_guidance=pg, image_guidance=ig,
                       input_prompt=prompt.id, output_prompt=prompt.id, save=False)
    datab.save_image(pimg, session_id=chat_id_4, user_id=2)


datab.close()


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

In [5]:
datab = Database()
datab.connect()

In [6]:
df_bert = datab.fetch_dataframe("SELECT * FROM bertscore_metrics")
df_guidance = datab.fetch_dataframe("SELECT * FROM guidance_metrics")
df_lpips = datab.fetch_dataframe("SELECT * FROM lpips_metrics")
df_func = datab.fetch_dataframe("SELECT * FROM functionality_metrics")
df_words = datab.fetch_dataframe("SELECT * FROM prompt_word_metrics")

print("BERTScore Metrics:\n", df_bert)
print("Guidance Metrics:\n", df_guidance)
print("LPIPS Metrics:\n", df_lpips)
print("Functionality Metrics:\n", df_func)
print("Prompt Word Metrics:\n", df_words)


BERTScore Metrics:
     id                             prompt_id  \
0    1  db1884e3-44b4-49d6-8bfa-becb25e4e4b1   
1    2  9bd03005-b16b-4b0f-8bcf-b5d5e4f9f6d6   
2    3  e770aef7-c08d-4560-9639-ab07bf01b020   
3    4  e35de6f3-6efe-4711-bd98-b851a54ffce1   
4    5  140581c9-329e-458a-af6f-bed92814bd24   
5    6  12f5d489-8064-451e-8a2c-421847dc0627   
6    7  46430905-f73c-4774-b47d-076d4c6a34a2   
7    8  ef5760ab-db06-4345-a82b-34eeb8f50e2c   
8    9  e9df08c3-54d3-4663-b4ec-771ff1229d72   
9   10  2aa5a23f-843b-4993-ab51-267cb7164852   
10  11  d6a779aa-d9b7-48ab-bffe-f5f813b34114   

                      previous_prompt_id  user_id  chat_id  depth  \
0                                   None        1        1      1   
1   db1884e3-44b4-49d6-8bfa-becb25e4e4b1        1        1      2   
2   9bd03005-b16b-4b0f-8bcf-b5d5e4f9f6d6        1        1      3   
3                                   None        1        3      1   
4   e35de6f3-6efe-4711-bd98-b851a54ffce1        1        3

### MMA Metrics Data Cheatsheet

#### Setup

Make sure you have imported and connected:

```python
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'app')))

print(sys.path)
from db.database import Database
import uuid

db = Database()
db.connect()
```

---

#### Metric Overview

| Metric Type       | Table Name              | What It Measures                                    |
| ----------------- | ----------------------- | --------------------------------------------------- |
| **BERTScore**     | `bertscore_metrics`     | Novelty of current prompt vs previous               |
| **Guidance**      | `guidance_metrics`      | Prompt/image guidance values used during generation |
| **LPIPS**         | `lpips_metrics`         | Visual change between images                        |
| **Functionality** | `functionality_metrics` | Suggestion/enhancement usage stats                  |
| **Prompt Words**  | `prompt_word_metrics`   | Word-level analysis of prompts                      |

---

#### Fetch All Metrics

```python
df_bert = db.fetch_all_bertscore_metrics()
df_guidance = db.fetch_all_guidance_metrics()
df_lpips = db.fetch_all_lpips_metrics()
df_func = db.fetch_all_functionality_metrics()
df_words = db.fetch_all_prompt_word_metrics()
```

In [7]:
datab.fetch_all_bertscore_metrics()

,id,prompt_id,previous_prompt_id,user_id,chat_id,depth,bert_novelty
0,1,db1884e3-44b4-49d6-8bfa-becb25e4e4b1,None,1,1,1,1.000000
1,2,9bd03005-b16b-4b0f-8bcf-b5d5e4f9f6d6,db1884e3-44b4-49d6-8bfa-becb25e4e4b1,1,1,2,0.040591
2,3,e770aef7-c08d-4560-9639-ab07bf01b020,9bd03005-b16b-4b0f-8bcf-b5d5e4f9f6d6,1,1,3,0.103445
3,4,e35de6f3-6efe-4711-bd98-b851a54ffce1,None,1,3,1,1.000000
4,5,140581c9-329e-458a-af6f-bed92814bd24,e35de6f3-6efe-4711-bd98-b851a54ffce1,1,3,2,0.097047
5,6,12f5d489-8064-451e-8a2c-421847dc0627,140581c9-329e-458a-af6f-bed92814bd24,1,3,3,0.123476
6,7,46430905-f73c-4774-b47d-076d4c6a34a2,None,2,4,1,1.000000
7,8,ef5760ab-db06-4345-a82b-34eeb8f50e2c,46430905-f73c-4774-b47d-076d4c6a34a2,2,4,2,0.090737
8,9,e9df08c3-54d3-4663-b4ec-771ff1229d72,ef5760ab-db06-4345-a82b-34eeb8f50e2c,2,4,3,0.145250
9,10,2aa5a23f-843b-4993-ab51-267cb7164852,e9df08c3-54d3-4663-b4ec-771ff1229d72,2,4,4,0.153407


In [8]:
datab.fetch_all_guidance_metrics()

,id,prompt_id,user_id,chat_id,depth,prompt_guidance,image_guidance
0,1,db1884e3-44b4-49d6-8bfa-becb25e4e4b1,1,1,1,6.0,0.40
1,2,9bd03005-b16b-4b0f-8bcf-b5d5e4f9f6d6,1,1,2,7.0,0.50
2,3,e35de6f3-6efe-4711-bd98-b851a54ffce1,1,3,1,5.5,0.30
3,4,12f5d489-8064-451e-8a2c-421847dc0627,1,3,3,6.5,0.45
4,5,140581c9-329e-458a-af6f-bed92814bd24,1,3,2,6.5,0.45
5,6,46430905-f73c-4774-b47d-076d4c6a34a2,2,4,1,6.0,0.40
6,7,ef5760ab-db06-4345-a82b-34eeb8f50e2c,2,4,2,6.5,0.45
7,8,e9df08c3-54d3-4663-b4ec-771ff1229d72,2,4,3,7.0,0.50
8,9,2aa5a23f-843b-4993-ab51-267cb7164852,2,4,4,7.5,0.55


In [9]:
datab.fetch_all_lpips_metrics()

,id,image_id,previous_image_id,user_id,chat_id,depth,lpips
0,1,439e5b70-7e9b-4b57-8279-1f3bd14d20c8,None,1,1,1,0.0
1,2,cf22a2b9-1a1b-47e4-916b-d1e66d328501,439e5b70-7e9b-4b57-8279-1f3bd14d20c8,1,1,2,0.0
2,3,eff56250-14f8-4149-911e-ae8444bac322,None,1,3,1,0.0
3,4,db394365-8dda-4e12-959b-2d33d514e98e,None,1,3,3,0.0
4,5,44f1927a-3d02-4bf3-b367-c301c12dceeb,eff56250-14f8-4149-911e-ae8444bac322,1,3,2,0.0
5,6,cc8f1566-4893-4693-b14f-8bdf174b792b,None,2,4,1,0.0
6,7,79997466-bea4-4cbe-80d4-dd549cac8ed0,cc8f1566-4893-4693-b14f-8bdf174b792b,2,4,2,0.0
7,8,a464f66a-aac4-4b42-bed2-954887af354c,79997466-bea4-4cbe-80d4-dd549cac8ed0,2,4,3,0.0
8,9,dca60c13-0d63-49ab-bd7b-b4f0bda91fdd,a464f66a-aac4-4b42-bed2-954887af354c,2,4,4,0.0


In [10]:

datab.fetch_all_functionality_metrics()

,id,user_id,chat_id,used_suggestion_pct,used_enhancement_pct,used_both_pct,no_ai_pct
0,3,1,1,0.3333,0.3333,0.3333,0.6667
1,6,1,3,0.3333,0.0000,0.0000,0.6667
2,11,2,4,0.2000,0.2000,0.0000,0.6000


In [11]:

datab.fetch_all_prompt_word_metrics()

,id,prompt_id,user_id,chat_id,depth,full_text,word_count,relevant_words
0,1,db1884e3-44b4-49d6-8bfa-becb25e4e4b1,1,1,1,Generate a bottle of perfume,5,"generate,bottle,perfume"
1,2,9bd03005-b16b-4b0f-8bcf-b5d5e4f9f6d6,1,1,2,Generate a bottle of perfume in a desert,8,"generate,bottle,perfume,desert"
2,3,e770aef7-c08d-4560-9639-ab07bf01b020,1,1,3,A perfume bottle in a desert with glowing suns...,10,"perfume,bottle,desert,glowing,sunset,hues"
3,4,e35de6f3-6efe-4711-bd98-b851a54ffce1,1,3,1,Design a futuristic drone,4,"design,futuristic,drone"
4,5,140581c9-329e-458a-af6f-bed92814bd24,1,3,2,Add neon lighting to the drone,6,"add,neon,lighting,drone"
5,6,12f5d489-8064-451e-8a2c-421847dc0627,1,3,3,Place the drone in a cyberpunk city,7,"place,drone,cyberpunk,city"
6,7,46430905-f73c-4774-b47d-076d4c6a34a2,2,4,1,A spaceship flying through clouds,5,"spaceship,flying,clouds"
7,8,ef5760ab-db06-4345-a82b-34eeb8f50e2c,2,4,2,Add sunlight breaking through clouds,5,"add,sunlight,breaking,clouds"
8,9,e9df08c3-54d3-4663-b4ec-771ff1229d72,2,4,3,Include a fleet of ships in the distance,8,"include,fleet,ships,distance"
9,10,2aa5a23f-843b-4993-ab51-267cb7164852,2,4,4,Make them metallic,3,"make,metallic"


---

#### Fetch by User

```python
user_id = 1

df_bert = db.fetch_bertscore_by_user(user_id)
df_guidance = db.fetch_guidance_by_user(user_id)
df_lpips = db.fetch_lpips_by_user(user_id)
df_func = db.fetch_functionality_by_user(user_id)
df_words = db.fetch_prompt_word_metrics_by_user(user_id)
```

In [12]:
user_id = 1

datab.fetch_bertscore_by_user(user_id)

,id,prompt_id,previous_prompt_id,user_id,chat_id,depth,bert_novelty
0,1,db1884e3-44b4-49d6-8bfa-becb25e4e4b1,None,1,1,1,1.000000
1,2,9bd03005-b16b-4b0f-8bcf-b5d5e4f9f6d6,db1884e3-44b4-49d6-8bfa-becb25e4e4b1,1,1,2,0.040591
2,3,e770aef7-c08d-4560-9639-ab07bf01b020,9bd03005-b16b-4b0f-8bcf-b5d5e4f9f6d6,1,1,3,0.103445
3,4,e35de6f3-6efe-4711-bd98-b851a54ffce1,None,1,3,1,1.000000
4,5,140581c9-329e-458a-af6f-bed92814bd24,e35de6f3-6efe-4711-bd98-b851a54ffce1,1,3,2,0.097047
5,6,12f5d489-8064-451e-8a2c-421847dc0627,140581c9-329e-458a-af6f-bed92814bd24,1,3,3,0.123476


In [13]:
datab.fetch_guidance_by_user(user_id)

,id,prompt_id,user_id,chat_id,depth,prompt_guidance,image_guidance
0,1,db1884e3-44b4-49d6-8bfa-becb25e4e4b1,1,1,1,6.0,0.40
1,2,9bd03005-b16b-4b0f-8bcf-b5d5e4f9f6d6,1,1,2,7.0,0.50
2,3,e35de6f3-6efe-4711-bd98-b851a54ffce1,1,3,1,5.5,0.30
3,4,12f5d489-8064-451e-8a2c-421847dc0627,1,3,3,6.5,0.45
4,5,140581c9-329e-458a-af6f-bed92814bd24,1,3,2,6.5,0.45


In [14]:
datab.fetch_lpips_by_user(user_id)

,id,image_id,previous_image_id,user_id,chat_id,depth,lpips
0,1,439e5b70-7e9b-4b57-8279-1f3bd14d20c8,None,1,1,1,0.0
1,2,cf22a2b9-1a1b-47e4-916b-d1e66d328501,439e5b70-7e9b-4b57-8279-1f3bd14d20c8,1,1,2,0.0
2,3,eff56250-14f8-4149-911e-ae8444bac322,None,1,3,1,0.0
3,4,db394365-8dda-4e12-959b-2d33d514e98e,None,1,3,3,0.0
4,5,44f1927a-3d02-4bf3-b367-c301c12dceeb,eff56250-14f8-4149-911e-ae8444bac322,1,3,2,0.0


In [15]:
datab.fetch_functionality_by_user(user_id)

,id,user_id,chat_id,used_suggestion_pct,used_enhancement_pct,used_both_pct,no_ai_pct
0,3,1,1,0.3333,0.3333,0.3333,0.6667
1,6,1,3,0.3333,0.0000,0.0000,0.6667


In [16]:
datab.fetch_prompt_word_metrics_by_user(user_id)

,id,prompt_id,user_id,chat_id,depth,full_text,word_count,relevant_words
0,1,db1884e3-44b4-49d6-8bfa-becb25e4e4b1,1,1,1,Generate a bottle of perfume,5,"generate,bottle,perfume"
1,2,9bd03005-b16b-4b0f-8bcf-b5d5e4f9f6d6,1,1,2,Generate a bottle of perfume in a desert,8,"generate,bottle,perfume,desert"
2,3,e770aef7-c08d-4560-9639-ab07bf01b020,1,1,3,A perfume bottle in a desert with glowing suns...,10,"perfume,bottle,desert,glowing,sunset,hues"
3,4,e35de6f3-6efe-4711-bd98-b851a54ffce1,1,3,1,Design a futuristic drone,4,"design,futuristic,drone"
4,5,140581c9-329e-458a-af6f-bed92814bd24,1,3,2,Add neon lighting to the drone,6,"add,neon,lighting,drone"
5,6,12f5d489-8064-451e-8a2c-421847dc0627,1,3,3,Place the drone in a cyberpunk city,7,"place,drone,cyberpunk,city"


---

#### Fetch by Chat

```python
chat_id = 2

df_bert = db.fetch_bertscore_by_chat(chat_id)
df_guidance = db.fetch_guidance_by_chat(chat_id)
df_lpips = db.fetch_lpips_by_chat(chat_id)
df_func = db.fetch_functionality_by_chat(chat_id)
df_words = db.fetch_prompt_word_metrics_by_chat(chat_id)
```



In [17]:
chat_id = 1

datab.fetch_bertscore_by_chat(chat_id)

,id,prompt_id,previous_prompt_id,user_id,chat_id,depth,bert_novelty
0,1,db1884e3-44b4-49d6-8bfa-becb25e4e4b1,None,1,1,1,1.000000
1,2,9bd03005-b16b-4b0f-8bcf-b5d5e4f9f6d6,db1884e3-44b4-49d6-8bfa-becb25e4e4b1,1,1,2,0.040591
2,3,e770aef7-c08d-4560-9639-ab07bf01b020,9bd03005-b16b-4b0f-8bcf-b5d5e4f9f6d6,1,1,3,0.103445


In [18]:
datab.fetch_guidance_by_chat(chat_id)

,id,prompt_id,user_id,chat_id,depth,prompt_guidance,image_guidance
0,1,db1884e3-44b4-49d6-8bfa-becb25e4e4b1,1,1,1,6.0,0.4
1,2,9bd03005-b16b-4b0f-8bcf-b5d5e4f9f6d6,1,1,2,7.0,0.5


In [19]:
datab.fetch_lpips_by_chat(chat_id)

,id,image_id,previous_image_id,user_id,chat_id,depth,lpips
0,1,439e5b70-7e9b-4b57-8279-1f3bd14d20c8,None,1,1,1,0.0
1,2,cf22a2b9-1a1b-47e4-916b-d1e66d328501,439e5b70-7e9b-4b57-8279-1f3bd14d20c8,1,1,2,0.0


In [20]:
datab.fetch_functionality_by_chat(chat_id)

,id,user_id,chat_id,used_suggestion_pct,used_enhancement_pct,used_both_pct,no_ai_pct
0,3,1,1,0.3333,0.3333,0.3333,0.6667


In [21]:
datab.fetch_functionality_by_chat(3)

,id,user_id,chat_id,used_suggestion_pct,used_enhancement_pct,used_both_pct,no_ai_pct
0,6,1,3,0.3333,0.0,0.0,0.6667


In [22]:
datab.fetch_prompt_word_metrics_by_chat(chat_id)

,id,prompt_id,user_id,chat_id,depth,full_text,word_count,relevant_words
0,1,db1884e3-44b4-49d6-8bfa-becb25e4e4b1,1,1,1,Generate a bottle of perfume,5,"generate,bottle,perfume"
1,2,9bd03005-b16b-4b0f-8bcf-b5d5e4f9f6d6,1,1,2,Generate a bottle of perfume in a desert,8,"generate,bottle,perfume,desert"
2,3,e770aef7-c08d-4560-9639-ab07bf01b020,1,1,3,A perfume bottle in a desert with glowing suns...,10,"perfume,bottle,desert,glowing,sunset,hues"


---

#### Column Hints

##### `bertscore_metrics`

* `bert_novelty`: float (0–1)
* `prompt_id`, `previous_prompt_id`

##### `guidance_metrics`

* `prompt_guidance`, `image_guidance`: floats
* `depth`, `prompt_id`

##### `lpips_metrics`

* `lpips`: float (0–1, lower = more similar)
* `image_id`, `previous_image_id`

##### `functionality_metrics`

* `used_suggestion_pct`, `used_enhancement_pct`, `used_both_pct`, `no_ai_pct`: % as floats
* One row per (user, chat)

##### `prompt_word_metrics`

* `full_text`: full prompt
* `relevant_words`: comma-separated words
* `word_count`: number of words after stopword removal


#### stuff for testing the inserting (insert_*)

In [ ]:
datab = Database()
datab.connect()

In [ ]:
# Clears all previous database entries
datab.reset_tables()

In [ ]:
# --- Step 1: Create Users ---
datab.insert_user("first_test_user", "password123")
datab.insert_user("second_test_user", "password456")

In [ ]:
# --- Step 2: Create Chats ---
datab.insert_chat("First Test Chat", 1)
datab.insert_chat("Second Test Chat", 2)

chat_id_1=1
chat_id_2=2

# --- Step 3: Create Prompts ---
prompt_id_0 = str(uuid.uuid4())
prompt_id_1 = str(uuid.uuid4())
prompt_id_2 = str(uuid.uuid4())
prompt_id_3 = str(uuid.uuid4())
prompt_id_4 = str(uuid.uuid4())

# Inputs for insert_prompt function:
# prompt_id, chat_id, user_id, prompt, depth, used_suggestion=False, modified_suggestion=False, suggestion_used=None, is_enhanced=False, enhanced_prompt=None, image_in_id=None, images_out=None

# note: as it stands Gilian's design expects each prompt_id to be a globally unique ID, such as a UUID.
# this is because the code says "CREATE TABLE IF NOT EXISTS prompts (id TEXT PRIMARY KEY,...)"
# we can maybe create a composite key (e.g., PRIMARY KEY (chat_id, id)) so that we distinguish them by chat/user
datab.insert_prompt(
    id=prompt_id_0, chat_id=chat_id_1, user_id=1,
    prompt="Generate a bottle of perfume", depth=1
)

datab.insert_prompt(
    id=prompt_id_1, chat_id=chat_id_1, user_id=1,
    prompt="Generate a bottle of perfume in a desert", depth=2
)

datab.insert_prompt(
    id=prompt_id_2, chat_id=chat_id_1, user_id=1,
    prompt="Add sunset lighting to the perfume bottle scene", depth=3,
    used_suggestion=True, suggestion_used="Try sunset colors", is_enhanced=True,
    enhanced_prompt="A perfume bottle in a desert with glowing sunset hues"
)

datab.insert_prompt(
    id=prompt_id_3, chat_id=chat_id_2, user_id=2,
    prompt="Design a stylish water bottle sticker", depth=1,
    used_suggestion=True, modified_suggestion=True, suggestion_used="Urban aesthetic",
    is_enhanced=True, enhanced_prompt="Water bottle sticker with urban graffiti"
)

datab.insert_prompt(
    id=prompt_id_4, chat_id=chat_id_2, user_id=2,
    prompt="Make it into a metallic sticker", depth=2,
    is_enhanced=False
)

# As can be seen in the code below, the input_prompt_id cannot be a str
# if not isinstance(input_prompt_id, str):
#     raise ValueError("input_prompt_id must be an integer")

# --- Step 4: Create Images ---
image_id_1 = str(uuid.uuid4())
image_id_2 = str(uuid.uuid4())
image_id_3 = str(uuid.uuid4())
image_id_4 = str(uuid.uuid4())

# Inputs for insert_image function:
# image_id: str, user_id: int, chat_id: int, prompt_guidance: float, image_guidance: float, path: str, input_prompt_id: str, output_prompt_id: str,
datab.insert_image(
    id=image_id_1, user_id=1, chat_id=chat_id_1,
    prompt_guidance=6.0, image_guidance=0.4,
    path="app/db/images/sauvage.png",
    input_prompt_id=prompt_id_1, output_prompt_id=prompt_id_1
)

datab.insert_image(
    id=image_id_2, user_id=1, chat_id=chat_id_1,
    prompt_guidance=7.0, image_guidance=0.5,
    path="app/db/images/sauvage_desert.png",
    input_prompt_id=prompt_id_2, output_prompt_id=prompt_id_2
)

datab.insert_image(
    id=image_id_3, user_id=2, chat_id=chat_id_2,
    prompt_guidance=5.5, image_guidance=0.3,
    path="app/db/images/test_image2.jpg",
    input_prompt_id=prompt_id_3, output_prompt_id=prompt_id_3
)

datab.insert_image(
    id=image_id_4, user_id=2, chat_id=chat_id_2,
    prompt_guidance=6.2, image_guidance=0.2,
    path="app/db/images/test_image4.jpg",
    input_prompt_id=prompt_id_4, output_prompt_id=prompt_id_4
)

In [ ]:
with Database() as data_base:  # adjust path if needed
    print("=== USERS ===")
    print(data_base.fetch_all_users())
    
    print("=== CHATS ===")
    print(data_base.fetch_all_chats())
    
    print("=== PROMPTS ===")
    print(data_base.fetch_all_prompts())
    
    print("=== IMAGES ===")
    print(data_base.fetch_all_images())

In [ ]:
data_base = Database()
data_base.connect()

In [ ]:
current_prompt_df = data_base.fetch_prompt_by_id('8dd6a2f4-6622-4a63-a093-8eac09b07b0b', pandas=True)

In [ ]:
if not current_prompt_df.empty:
    depth = current_prompt_df.iloc[0]["depth"]
else:
    depth = 0

print(depth)

In [ ]:
prior_images = data_base.fetch_images_by_chat(chat_id_1, pandas=True)
# print(f"prior_images1: {prior_images}")
prior_images = prior_images.merge(data_base.fetch_all_prompts(), how="left", left_on="output_prompt_id", right_on="id")
# print(f"prior_images2: {prior_images}")
prior_images = prior_images[prior_images["depth"] == depth - 1]
print(f"prior_images3: {prior_images}")

In [ ]:
previous_image_id = None
previous_image_path = None
if not prior_images.empty:
        previous_image_id = prior_images.iloc[0]["id_x"]  # id_x = image.id
        print(previous_image_id)
        previous_image_path = prior_images.iloc[0]["path"]
        print(previous_image_path)


In [ ]:
current_images = data_base.fetch_images_by_chat(chat_id_1, pandas=True)
# print(f"prior_images1: {prior_images}")
current_images = current_images.merge(data_base.fetch_all_prompts(), how="left", left_on="output_prompt_id", right_on="id")
current_images = current_images[current_images["depth"] == depth]
print(f"current_images3: {current_images}")
print(current_images.iloc[0]["id_x"])
print(current_images.iloc[0]["path"])

In [ ]:
from modules.metrics import get_or_compute_bertscore, extract_relevant_words, get_or_compute_lpips

In [ ]:
lpips_score = None
if previous_image_path:
    lpips_score = get_or_compute_lpips("6122bcfc-fb9d-4f2c-afdf-bd93192b4b22", "images/sauvage_desert.png", "images/sauvage.png")
else:
    lpips_score = None

print(str(lpips_score))

In [ ]:
ls

In [ ]:
data_base = Database()
data_base.connect()

In [ ]:

datab.fetch_all_prompts()

In [ ]:
# Default for reading to pandas dataframe is True, change to False if you want to fetch without pandas

datab.fetch_all_users(pandas=True)

In [ ]:

# Default for reading to pandas dataframe is True, change to False if you want to fetch without pandas

datab.fetch_all_images(pandas=True)

In [ ]:
datab.close()